# MLOps Zoomcamp – Week 1 Homework

**Owner:** Martín Berns  
**Cohort:** 2025  

This notebook contains the solutions to the Week 1 homework from the  
[DataTalksClub MLOps Zoomcamp](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2025/01-intro/homework.md).

In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [4]:
df_jan = pd.read_parquet('/workspaces/mlops-zoomcamp/01-intro/homework/data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('/workspaces/mlops-zoomcamp/01-intro/homework/data/yellow_tripdata_2023-02.parquet')

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [5]:
len(df_jan.columns)

19

### Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [6]:
df_jan

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [7]:
df_jan.tpep_dropoff_datetime = pd.to_datetime(df_jan.tpep_dropoff_datetime)
df_jan.tpep_pickup_datetime = pd.to_datetime(df_jan.tpep_pickup_datetime)

df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

df_feb.tpep_dropoff_datetime = pd.to_datetime(df_feb.tpep_dropoff_datetime)
df_feb.tpep_pickup_datetime = pd.to_datetime(df_feb.tpep_pickup_datetime)

df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
pd.options.display.float_format = '{:.2f}'.format
df_jan['duration'].describe()

count   3066766.00
mean         15.67
std          42.59
min         -29.20
25%           7.12
50%          11.52
75%          18.30
max       10029.18
Name: duration, dtype: float64

### Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%


In [9]:
n_before = len(df_jan)

df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

n_after = len(df_jan)
kept_fraction = n_after / n_before * 100

print(kept_fraction)

98.1220282212598


### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [10]:
categorical = ['PULocationID', 'DOLocationID']

df_jan.loc[:, categorical] = df_jan[categorical].astype(str)
df_feb.loc[:, categorical] = df_feb[categorical].astype(str)

dv = DictVectorizer()

train_dicts_jan = df_jan[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts_jan)

train_dicts_feb = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(train_dicts_feb)

/tmp/ipykernel_41504/3991129478.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan.loc[:, categorical] = df_jan[categorical].astype(str)


In [11]:
X_train.shape[1]

515

### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64


In [12]:
target = 'duration'
y_train = df_jan[target].values
y_val = df_feb[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)


rmse = lambda mse: mse**0.5
train_rmse = rmse(mean_squared_error(y_train, y_train_pred))

print(f'RMSE train: {train_rmse}')

RMSE train: 7.6492610279057605


### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81


In [14]:
y_val_pred = lr.predict(X_val)

val_rmse = rmse(mean_squared_error(y_val, y_val_pred))

print(f'RMSE val: {val_rmse}')

RMSE val: 7.81183265470218
